In [1]:
!pip install pyomo -q
!pip install mistralai -q
!wget -N -q "https://matematica.unipv.it/gualandi/solvers/ipopt-linux64.zip"
!unzip -o -q ipopt-linux64

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.8/12.8 MB 19.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.6/49.6 kB 3.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.0/75.0 kB 2.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 145.0/145.0 kB 1.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 6.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 6.2 MB/s eta 0:00:00


In [2]:
import os
from mistralai.client import MistralClient
from mistralai.models.chat_completion import ChatMessage
from IPython.display import display, Markdown, Latex
from datetime import datetime

In [3]:
API_KEY = ''

MODEL_ID = 'open-mixtral-8x22b'
MODEL_SEED = 2
MODEL_TEMPERATURE = 0.7

SYSTEM_PROMPT_1 = """Please formulate a mathematical optimization model for this problem.
It is important that you do this by following these steps:

1. Define parameters and variables
2. Define the objective function
3. Define the constraints

Please note that each answer is always dedicated to one step. Therefore, do not provide extra information than what is asked. The answers will be manually combined by the user to create the full model."""
SYSTEM_PROMPT_2 = """Please write a python pyomo code for this optimization problem. Use sample data where needed. Indicate where you use sample data."""
USER_PROMPT_1 = """We are now at the first step."""
USER_PROMPT_2 = """We are now at the second step."""
USER_PROMPT_3 = """We are now at the third step."""
PROBLEM_DESCRIPTION = """"A firm that packs refreshments and beers, situated in the province of Valencia (Spain) employs the same syrup to produce its 1.5 l COLI and PEPSA products on its S1 production line. Once processed, each hectolitre of syrup produces 40 units of the 1.5 l COLI product and 20 units of the 1.5 l PEPSA product. If X1 units of the 1.5 l COLI product and X2 units of the 1.5 l PEPSA product are produced, the firm estimates that the daily income obtained in dollars would be given by the following function:
49000 times X1 minus X1 squared plus 30 times X2 minus two times X2 squared.
It costs 150 dollars to buy and process each hectolitre of syrup. The S1 packaging line has a net capacity of producing 7100 1.5 l product units every hour. The firm works 5 days a week in 8h shifts. Given its weekly target coverage, the firm is committed to produce at least half the amount of PEPSA than COLI. Although priority orders tend to amend its production planning, the firm wishes to have a basic product planning that optimises its daily profits."""



client = MistralClient(api_key=API_KEY)

print(f'Time of execution: {datetime.now()}')

Time of execution: 2024-06-14 08:10:27.914475


## Step 1 - Generate Mathematical Formulation

In [4]:
messages_1 = [
    ChatMessage(role="system", content=SYSTEM_PROMPT_1),
    ChatMessage(role="user", content=PROBLEM_DESCRIPTION),
    ChatMessage(role="user", content=USER_PROMPT_1)
]

response_1 = client.chat(
    model=MODEL_ID,
    messages=messages_1,
    random_seed=MODEL_SEED,
    temperature=MODEL_TEMPERATURE
)

response_1_text = response_1.choices[0].message.content

In [5]:
Markdown(response_1_text)

Step 1: Define parameters and variables

Parameters:
- $C_1 = 49,000$: Coefficient for X1 in the income function
- $C_2 = 30$: Coefficient for X2 in the income function
- $P_{COL} = 40$: Number of 1.5 l COLI units produced per hectolitre of syrup
- $P_{PEP} = 20$: Number of 1.5 l PEPSA units produced per hectolitre of syrup
- $H = 7100$: Net hourly production capacity of the S1 line in 1.5 l units
- $T = 40$: Number of hours worked per week (5 days a week, 8 hour shifts)
- $S = 150$: Cost to buy and process each hectolitre of syrup

Variables:
- $X_1$: Number of 1.5 l COLI units produced
- $X_2$: Number of 1.5 l PEPSA units produced

The variables $X_1$ and $X_2$ determine the production quantities for the 1.5 l COLI and PEPSA products, respectively.

In [6]:
print(response_1_text)

Step 1: Define parameters and variables

Parameters:
- $C_1 = 49,000$: Coefficient for X1 in the income function
- $C_2 = 30$: Coefficient for X2 in the income function
- $P_{COL} = 40$: Number of 1.5 l COLI units produced per hectolitre of syrup
- $P_{PEP} = 20$: Number of 1.5 l PEPSA units produced per hectolitre of syrup
- $H = 7100$: Net hourly production capacity of the S1 line in 1.5 l units
- $T = 40$: Number of hours worked per week (5 days a week, 8 hour shifts)
- $S = 150$: Cost to buy and process each hectolitre of syrup

Variables:
- $X_1$: Number of 1.5 l COLI units produced
- $X_2$: Number of 1.5 l PEPSA units produced

The variables $X_1$ and $X_2$ determine the production quantities for the 1.5 l COLI and PEPSA products, respectively.


In [7]:
messages_2 = [
    ChatMessage(role="system", content=SYSTEM_PROMPT_1),
    ChatMessage(role="user", content=PROBLEM_DESCRIPTION),
    ChatMessage(role="user", content=USER_PROMPT_1),
    ChatMessage(role="assistant", content=response_1_text),
    ChatMessage(role="user", content=USER_PROMPT_2),
]

response_2 = client.chat(
    model=MODEL_ID,
    messages=messages_2,
    random_seed=MODEL_SEED,
    temperature=MODEL_TEMPERATURE
)

response_2_text = response_2.choices[0].message.content

In [8]:
Markdown(response_2_text)

Step 2: Define the objective function

The objective function represents the daily profit obtained from selling the 1.5 l COLI and PEPSA products. The profit is calculated by subtracting the syrup processing cost from the total income.

The income function is given by:
- $49,000 \times X_1 - X_1^2 + 30 \times X_2 - 2 \times X_2^2$

The syrup processing cost is calculated by multiplying the cost per hectolitre ($150) by the total number of hectolitres required to produce $X_1$ and $X_2$. Since each hectolitre produces $40$ COLI units and $20$ PEPSA units, the total number of hectolitres required can be expressed as:
- $\frac{X_1}{40} + \frac{X_2}{20}$

Therefore, the cost of processing syrup is:
- $150 \times (\frac{X_1}{40} + \frac{X_2}{20})$
- Simplifying, we get:
- $3.75X_1 + 7.5X_2$

The objective function, which is the daily profit, can be written as:
- $49,000 \times X_1 - X_1^2 + 30 \times X_2 - 2 \times X_2^2 - (3.75X_1 + 7.5X_2)$
- Simplifying, we get:
- $48,996.25X_1 - X_1^2 + 22.5X_2 - 2X_2^2$

The objective function is to maximize this daily profit.

In [9]:
print(response_2_text)

Step 2: Define the objective function

The objective function represents the daily profit obtained from selling the 1.5 l COLI and PEPSA products. The profit is calculated by subtracting the syrup processing cost from the total income.

The income function is given by:
- $49,000 \times X_1 - X_1^2 + 30 \times X_2 - 2 \times X_2^2$

The syrup processing cost is calculated by multiplying the cost per hectolitre ($150) by the total number of hectolitres required to produce $X_1$ and $X_2$. Since each hectolitre produces $40$ COLI units and $20$ PEPSA units, the total number of hectolitres required can be expressed as:
- $\frac{X_1}{40} + \frac{X_2}{20}$

Therefore, the cost of processing syrup is:
- $150 \times (\frac{X_1}{40} + \frac{X_2}{20})$
- Simplifying, we get:
- $3.75X_1 + 7.5X_2$

The objective function, which is the daily profit, can be written as:
- $49,000 \times X_1 - X_1^2 + 30 \times X_2 - 2 \times X_2^2 - (3.75X_1 + 7.5X_2)$
- Simplifying, we get:
- $48,996.25X_1 - X_1^2 +

In [10]:
messages_3 = [
    ChatMessage(role="system", content=SYSTEM_PROMPT_1),
    ChatMessage(role="user", content=PROBLEM_DESCRIPTION),
    ChatMessage(role="user", content=USER_PROMPT_1),
    ChatMessage(role="assistant", content=response_1_text),
    ChatMessage(role="user", content=USER_PROMPT_2),
    ChatMessage(role="assistant", content=response_2_text),
    ChatMessage(role="user", content=USER_PROMPT_3),
]

response_3 = client.chat(
    model=MODEL_ID,
    messages=messages_3,
    random_seed=MODEL_SEED,
    temperature=MODEL_TEMPERATURE
)

response_3_text = response_3.choices[0].message.content

In [11]:
Markdown(response_3_text)

Step 3: Define the constraints

There are three constraints for this problem:

1. Production capacity constraint: The total number of 1.5 l units produced (both COLI and PEPSA) must not exceed the hourly production capacity of the S1 line multiplied by the number of hours worked per week.

   - $X_1 + X_2 \leq 7100 \times 40$

2. Minimum production ratio constraint: The firm is committed to producing at least half the amount of PEPSA than COLI.

   - $X_2 \geq 0.5X_1$

3. Non-negativity constraints: The number of 1.5 l COLI and PEPSA units produced must be non-negative.

   - $X_1 \geq 0$
   - $X_2 \geq 0$

These constraints ensure that the production plan is feasible and meets the firm's requirements.

Now, you can combine all the steps to formulate the full mathematical optimization model:

Step 1: Define parameters and variables
Step 2: Define the objective function as $\max 48,996.25X_1 - X_1^2 + 22.5X_2 - 2X_2^2$
Step 3: Define the constraints as:

1. $X_1 + X_2 \leq 7100 \times 40$
2. $X_2 \geq 0.5X_1$
3. $X_1 \geq 0$
4. $X_2 \geq 0$

This mathematical optimization model can now be used to find the optimal production quantities for the 1.5 l COLI and PEPSA products that maximize the firm's daily profit while meeting the given constraints.

In [12]:
print(response_3_text)

Step 3: Define the constraints

There are three constraints for this problem:

1. Production capacity constraint: The total number of 1.5 l units produced (both COLI and PEPSA) must not exceed the hourly production capacity of the S1 line multiplied by the number of hours worked per week.

   - $X_1 + X_2 \leq 7100 \times 40$

2. Minimum production ratio constraint: The firm is committed to producing at least half the amount of PEPSA than COLI.

   - $X_2 \geq 0.5X_1$

3. Non-negativity constraints: The number of 1.5 l COLI and PEPSA units produced must be non-negative.

   - $X_1 \geq 0$
   - $X_2 \geq 0$

These constraints ensure that the production plan is feasible and meets the firm's requirements.

Now, you can combine all the steps to formulate the full mathematical optimization model:

Step 1: Define parameters and variables
Step 2: Define the objective function as $\max 48,996.25X_1 - X_1^2 + 22.5X_2 - 2X_2^2$
Step 3: Define the constraints as:

1. $X_1 + X_2 \leq 7100 \times 4

In [13]:
response_text = '\n'.join([response_1_text, response_2_text, response_3_text])

Markdown(response_text)

Step 1: Define parameters and variables

Parameters:
- $C_1 = 49,000$: Coefficient for X1 in the income function
- $C_2 = 30$: Coefficient for X2 in the income function
- $P_{COL} = 40$: Number of 1.5 l COLI units produced per hectolitre of syrup
- $P_{PEP} = 20$: Number of 1.5 l PEPSA units produced per hectolitre of syrup
- $H = 7100$: Net hourly production capacity of the S1 line in 1.5 l units
- $T = 40$: Number of hours worked per week (5 days a week, 8 hour shifts)
- $S = 150$: Cost to buy and process each hectolitre of syrup

Variables:
- $X_1$: Number of 1.5 l COLI units produced
- $X_2$: Number of 1.5 l PEPSA units produced

The variables $X_1$ and $X_2$ determine the production quantities for the 1.5 l COLI and PEPSA products, respectively.
Step 2: Define the objective function

The objective function represents the daily profit obtained from selling the 1.5 l COLI and PEPSA products. The profit is calculated by subtracting the syrup processing cost from the total income.

The income function is given by:
- $49,000 \times X_1 - X_1^2 + 30 \times X_2 - 2 \times X_2^2$

The syrup processing cost is calculated by multiplying the cost per hectolitre ($150) by the total number of hectolitres required to produce $X_1$ and $X_2$. Since each hectolitre produces $40$ COLI units and $20$ PEPSA units, the total number of hectolitres required can be expressed as:
- $\frac{X_1}{40} + \frac{X_2}{20}$

Therefore, the cost of processing syrup is:
- $150 \times (\frac{X_1}{40} + \frac{X_2}{20})$
- Simplifying, we get:
- $3.75X_1 + 7.5X_2$

The objective function, which is the daily profit, can be written as:
- $49,000 \times X_1 - X_1^2 + 30 \times X_2 - 2 \times X_2^2 - (3.75X_1 + 7.5X_2)$
- Simplifying, we get:
- $48,996.25X_1 - X_1^2 + 22.5X_2 - 2X_2^2$

The objective function is to maximize this daily profit.
Step 3: Define the constraints

There are three constraints for this problem:

1. Production capacity constraint: The total number of 1.5 l units produced (both COLI and PEPSA) must not exceed the hourly production capacity of the S1 line multiplied by the number of hours worked per week.

   - $X_1 + X_2 \leq 7100 \times 40$

2. Minimum production ratio constraint: The firm is committed to producing at least half the amount of PEPSA than COLI.

   - $X_2 \geq 0.5X_1$

3. Non-negativity constraints: The number of 1.5 l COLI and PEPSA units produced must be non-negative.

   - $X_1 \geq 0$
   - $X_2 \geq 0$

These constraints ensure that the production plan is feasible and meets the firm's requirements.

Now, you can combine all the steps to formulate the full mathematical optimization model:

Step 1: Define parameters and variables
Step 2: Define the objective function as $\max 48,996.25X_1 - X_1^2 + 22.5X_2 - 2X_2^2$
Step 3: Define the constraints as:

1. $X_1 + X_2 \leq 7100 \times 40$
2. $X_2 \geq 0.5X_1$
3. $X_1 \geq 0$
4. $X_2 \geq 0$

This mathematical optimization model can now be used to find the optimal production quantities for the 1.5 l COLI and PEPSA products that maximize the firm's daily profit while meeting the given constraints.

In [14]:
print(response_text)

Step 1: Define parameters and variables

Parameters:
- $C_1 = 49,000$: Coefficient for X1 in the income function
- $C_2 = 30$: Coefficient for X2 in the income function
- $P_{COL} = 40$: Number of 1.5 l COLI units produced per hectolitre of syrup
- $P_{PEP} = 20$: Number of 1.5 l PEPSA units produced per hectolitre of syrup
- $H = 7100$: Net hourly production capacity of the S1 line in 1.5 l units
- $T = 40$: Number of hours worked per week (5 days a week, 8 hour shifts)
- $S = 150$: Cost to buy and process each hectolitre of syrup

Variables:
- $X_1$: Number of 1.5 l COLI units produced
- $X_2$: Number of 1.5 l PEPSA units produced

The variables $X_1$ and $X_2$ determine the production quantities for the 1.5 l COLI and PEPSA products, respectively.
Step 2: Define the objective function

The objective function represents the daily profit obtained from selling the 1.5 l COLI and PEPSA products. The profit is calculated by subtracting the syrup processing cost from the total income.

Th

## Step 2 - Generate the Pyomo Code

In [15]:
messages_4 = [
    ChatMessage(role="system", content=SYSTEM_PROMPT_2),
    ChatMessage(role="user", content=response_text)
]

In [16]:
response_4 = client.chat(
    model=MODEL_ID,
    messages=messages_4,
    random_seed=MODEL_SEED,
    temperature=MODEL_TEMPERATURE
)

response_4_text = response_4.choices[0].message.content

In [17]:
Markdown(response_4_text)

Here is the Python code using Pyomo to solve the optimization problem:

```python
from pyomo.environ import *

# Step 1: Define parameters and variables
C1 = 49000
C2 = 30
P_COL = 40
P_PEP = 20
H = 7100
T = 40
S = 150

# Create a model
model = ConcreteModel()

# Define decision variables
model.X1 = Var(within=NonNegativeReals)
model.X2 = Var(within=NonNegativeReals)

# Step 2: Define the objective function
def obj_rule(model):
    return C1 * model.X1 - model.X1**2 + C2 * model.X2 - 2 * model.X2**2 - (S/P_COL * model.X1 + S/P_PEP * model.X2)

model.obj = Objective(rule=obj_rule, sense=maximize)

# Step 3: Define the constraints
def production_capacity_constraint_rule(model):
    return model.X1 + model.X2 <= H * T

model.production_capacity_constraint = Constraint(rule=production_capacity_constraint_rule)

def minimum_production_ratio_constraint_rule(model):
    return model.X2 >= 0.5 * model.X1

model.minimum_production_ratio_constraint = Constraint(rule=minimum_production_ratio_constraint_rule)

# Solve the model
solver = SolverFactory('glpk')
solver.solve(model)

# Print the results
print("X1 = ", model.X1.value)
print("X2 = ", model.X2.value)
```

In the code above, I used sample data as provided in the problem statement. The objective function and constraints are defined according to the problem statement. The `glpk` solver is used to solve the model. Finally, the optimal values of the decision variables `X1` and `X2` are printed.

In [18]:
print(response_4_text)

Here is the Python code using Pyomo to solve the optimization problem:

```python
from pyomo.environ import *

# Step 1: Define parameters and variables
C1 = 49000
C2 = 30
P_COL = 40
P_PEP = 20
H = 7100
T = 40
S = 150

# Create a model
model = ConcreteModel()

# Define decision variables
model.X1 = Var(within=NonNegativeReals)
model.X2 = Var(within=NonNegativeReals)

# Step 2: Define the objective function
def obj_rule(model):
    return C1 * model.X1 - model.X1**2 + C2 * model.X2 - 2 * model.X2**2 - (S/P_COL * model.X1 + S/P_PEP * model.X2)

model.obj = Objective(rule=obj_rule, sense=maximize)

# Step 3: Define the constraints
def production_capacity_constraint_rule(model):
    return model.X1 + model.X2 <= H * T

model.production_capacity_constraint = Constraint(rule=production_capacity_constraint_rule)

def minimum_production_ratio_constraint_rule(model):
    return model.X2 >= 0.5 * model.X1

model.minimum_production_ratio_constraint = Constraint(rule=minimum_production_ratio_const

### Code Executability

In [22]:
from pyomo.environ import *

# Step 1: Define parameters and variables
C1 = 49000
C2 = 30
P_COL = 40
P_PEP = 20
H = 7100
T = 40
S = 150

# Create a model
model = ConcreteModel()

# Define decision variables
model.X1 = Var(within=NonNegativeReals)
model.X2 = Var(within=NonNegativeReals)

# Step 2: Define the objective function
def obj_rule(model):
    return C1 * model.X1 - model.X1**2 + C2 * model.X2 - 2 * model.X2**2 - (S/P_COL * model.X1 + S/P_PEP * model.X2)

model.obj = Objective(rule=obj_rule, sense=maximize)

# Step 3: Define the constraints
def production_capacity_constraint_rule(model):
    return model.X1 + model.X2 <= H * T

model.production_capacity_constraint = Constraint(rule=production_capacity_constraint_rule)

def minimum_production_ratio_constraint_rule(model):
    return model.X2 >= 0.5 * model.X1

model.minimum_production_ratio_constraint = Constraint(rule=minimum_production_ratio_constraint_rule)

# Solve the model
solver = SolverFactory('ipopt')
solver.solve(model)

# Print the results
print(model.obj())
print("X1 = ", model.X1.value)
print("X2 = ", model.X2.value)

400289176.0419919
X1 =  16335.833333340022
X2 =  8167.9166666600495


### Solution Correctness